In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
pd.set_option('display.max_columns', 5000)

In [6]:
#compares a player with the mean of his position
def comparison(playerid, matchId, file, cols):
    total=pd.read_csv(file)
    m=total[cols].mean()
    print('The mean:')
    print(m)
    total=total[total['playerId']==playerid]
    total=total[total['matchId']==matchId]
    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    print('player:', dfw['player.name'].iloc[0])
    print(total[cols])
    

In [7]:
#returns the player using player and match id
def get_player(playerid, matchId):
    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    return dfw['player.name'].iloc[0]

In [8]:
#print the top 50 players according to the ranking
def bestplayers(df):
    for i in range (50):
        id=df['playerId'].iloc[i]
        matchid=df['matchId'].iloc[i]
        player=get_player(id,matchid)
        print('matchid:',matchid)
        print('playerid:', id)
        print(player)

In [9]:
#negative features are multiplied by -1 for the convention
loss_list= [
    'total.losses',
    'total.ownHalfLosses',
    'total.dangerousOwnHalfLosses'
]

In [50]:
#preprocess and filter the dataframe with the feature set 
def preprocessdf(mean, std, position):
    meandf=pd.read_csv(mean)
    stddf=pd.read_csv(std)
    pdf=pd.read_csv(position)
    meandf=meandf[meandf.columns[1:]]
    stddf=stddf[stddf.columns[1:]]
    '''
    pdf=pdf[pdf.columns[1:]]
    '''
    keep_columns=meandf.columns
    calc_col = [col for col in pdf.columns if col.startswith('total.') and col != 'total.minutesOnField']
    calc_filtered = [col for col in calc_col if col not in keep_columns]
    cpdf=pdf.drop(columns=calc_filtered)
    for w in loss_list:
        if(w in keep_columns):
            cpdf[w]*=-1
    return cpdf, meandf, stddf

In [11]:
#calculate the weighted sum of the data normalized to all clusters from the result
def resultcalculation(calc,mean,std,i, weight):
    t0=calc.copy()
    
    t0[mean.columns]-=mean.loc[i]
    
    t0[mean.columns]/=std.loc[i]

    vec=np.array(weight)
    calc_col = [col for col in calc.columns if col.startswith('total.') and col != 'total.minutesOnField']
    statsarray0=t0[calc_col].to_numpy()
    res0=np.dot(statsarray0, vec)
    return res0

In [118]:
meanpath='mean_std/mean_stv3clustering.csv'
stdpath='mean_std/std_stv3clustering.csv'
playerpath='norm_stats/normalized_striker.csv'

In [119]:
pd.read_csv(meanpath)

,Unnamed: 0,total.goals,total.shots,total.keyPasses,total.shotAssists,total.recoveries,total.opponentHalfRecoveries,total.losses,total.ownHalfLosses,total.xgShot,total.xgAssist,total.receivedPass,total.touchInBox,total.progressivePasses,total.shotsOnTarget
0,0,0.406524,0.637292,-0.272048,-0.232560,-0.553720,-0.465354,0.033788,0.157258,0.480186,-0.281279,0.016407,0.488706,-0.329428,0.612906
1,1,-0.320824,-0.442419,-0.131503,-0.073640,0.685936,0.496176,-0.599109,-0.622552,-0.406975,-0.159202,-0.044779,-0.395813,0.337852,-0.424786
2,2,-0.441474,-0.508442,-0.222245,-0.274371,-0.408421,-0.332208,0.795881,0.625220,-0.435311,-0.297816,-0.425898,-0.421798,-0.406984,-0.483749
3,3,0.017723,-0.071204,0.016445,-0.024496,-0.158881,-0.140521,-0.225830,-0.233655,-0.080834,-0.052462,-0.083287,-0.087067,-0.084544,-0.062303


In [120]:
calcdf, meandf, stddf=preprocessdf(meanpath, stdpath, playerpath)

In [135]:
calcdf

,Unnamed: 0,playerId,matchId,competitionId,seasonId,roundId,positions,total.minutesOnField,total.goals,total.shots,total.keyPasses,total.shotAssists,total.recoveries,total.opponentHalfRecoveries,total.losses,total.ownHalfLosses,total.xgShot,total.xgAssist,total.receivedPass,total.touchInBox,total.progressivePasses,total.shotsOnTarget,false 9,box,target,2nd runner
0,1,37840,2799513,103,21316,4418495,"[{'position': {'name': 'Attacking Midfielder',...",47,-0.505282,-0.102409,-0.505924,-0.699164,3.408461,3.857077,-0.473782,0.328131,-0.526471,-0.486072,1.032409,-0.446696,-0.983922,-0.770571,14.778226,3.861847,25.544057,16.431685
1,2,8717,2799513,103,21316,4418495,"[{'position': {'name': 'Striker', 'code': 'cf'...",94,0.863499,0.367935,0.872224,0.173948,0.584133,0.156141,-0.668919,0.328131,1.255589,2.308808,0.706425,0.179522,1.310946,0.807388,38.998791,27.903555,79.383309,61.722692
2,3,234359,2799513,103,21316,4418495,"[{'position': {'name': 'Striker', 'code': 'cf'...",83,-0.505282,-0.510418,1.054870,0.289662,1.366778,1.018206,-1.498843,-2.507384,-0.589231,0.769128,-0.590965,-1.072914,1.095284,-0.770571,-7.835312,-13.879473,-13.611667,-4.481915
3,4,69410,2799514,103,21316,4418495,"[{'position': {'name': 'Striker', 'code': 'cf'...",96,-0.505282,-1.043097,-0.505924,-0.699164,-0.376923,-0.473532,-0.225794,0.738115,-0.722301,-0.486072,-1.256270,-1.072914,-0.983922,-0.770571,-18.505400,-34.665500,-18.870310,-27.995189
4,5,33062,2799514,103,21316,4418495,"[{'position': {'name': 'Striker', 'code': 'cf'...",84,-0.505282,0.009578,-0.505924,0.277890,0.281526,0.993257,0.511199,0.681936,0.198103,-0.108602,0.491690,0.328621,0.556918,-0.770571,11.415165,-0.708183,23.354490,15.978147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572,2277,54359,5414332,103,187004,4427756,"[{'position': {'name': 'Striker', 'code': 'cf'...",57,-0.505282,-1.043097,-0.505924,-0.699164,1.806356,2.991361,-1.281720,0.469049,-0.722301,-0.486072,0.216496,-1.072914,0.529886,-0.770571,3.801727,-11.744172,8.347025,-2.413546
1573,2278,73839,5414332,103,187004,4427756,"[{'position': {'name': 'Striker', 'code': 'cf'...",39,-0.505282,-1.043097,-0.505924,-0.699164,0.970366,1.895897,-1.034177,-0.804636,-0.722301,-0.486072,-1.051659,-1.072914,-0.983922,-0.770571,-13.784381,-24.983529,-16.311834,-17.522077
1574,2279,285867,5414332,103,187004,4427756,"[{'position': {'name': 'Striker', 'code': 'cf'...",74,1.233440,-0.445633,-0.505924,-0.699164,1.093021,1.273090,-1.955774,-2.439773,-0.548171,-0.486072,0.356947,-0.675181,0.765140,0.231646,-3.589929,-7.268088,5.479646,11.744644
1575,2280,539358,5414332,103,187004,4427756,"[{'position': {'name': 'Attacking Midfielder',...",39,-0.505282,1.224202,-0.505924,-0.699164,-1.298752,-1.077504,-0.093513,0.163364,-0.439100,-0.486072,0.519751,1.191105,0.122322,-0.770571,-8.864410,-15.585951,-8.259228,-1.554066


In [124]:
#weight vector for each player profile
iv_def_weights=[1,2,1,4,1,4,1,4,2,1,4,1,2,4,1,2,4,1,0,1,1]
iv_hyb_weights=[1,2,1,4,1,4,1,4,2,1,4,1,4,1,1,2,4,1,0,1,2]
iv_off_weights=[1,4,1,4,1,1,1,4,4,1,4,1,4,1,2,2,4,2,2,1,2]
wi_pla_weights=[1,1,4,1,1,4,2,1,4,2,4,4,2,4,4]
wi_dri_weights=[1,1,4,1,4,2,4,1,4,4,4,2,4,4,2]
wi_run_weights=[1,1,4,1,2,2,4,1,4,4,4,1,4,4,4]
st_run_weights=[4,1,0,2,2,4,2,1,4,4,4,4,1,4]
st_goa_weights=[4,1,0,2,2,4,2,4,4,4,2,2,1,4]
st_box_weights=[4,2,0,2,2,2,1,1,4,2,2,4,1,4]
st_fa9_weights=[4,2,0,2,2,4,2,4,4,4,2,2,2,4]
mi_dpl_weights=[1,4,0,1,4,4,1,4,1,2,1,1,2,4,4,1,1,2,4,0,2,4,0,1]
mi_opl_weights=[0,0,4,0,0,4,1,4,0,0,4,0,0,0,2,1,1,0,0,4,4,2,2,0]
mi_def_weights=[1,4,0,1,2,2,1,4,1,2,1,1,1,4,4,2,4,2,4,0,2,4,0,2]
mi_off_weights=[0,0,4,1,4,1,1,4,0,0,1,0,0,2,2,2,4,0,0,2,4,2,2,4]
lb_def_weights=[1,2,1,4,1,4,1,4,1,1,1,2,4,1,4,1,2,4,1,1,4,1,0,0,1,1]
lb_off_weights=[1,4,1,2,1,2,2,4,2,4,1,4,2,1,2,1,2,2,4,2,2,1,4,4,1,1]
lb_run_weights=[1,4,1,4,1,4,1,2,1,4,1,4,2,1,4,1,2,2,4,1,2,1,4,4,1,2]
unitlist=[1]*len(mi_def_weights)

In [122]:
featuredf=pd.DataFrame(columns=meandf.columns)
featuredf

,total.goals,total.shots,total.keyPasses,total.shotAssists,total.recoveries,total.opponentHalfRecoveries,total.losses,total.ownHalfLosses,total.xgShot,total.xgAssist,total.receivedPass,total.touchInBox,total.progressivePasses,total.shotsOnTarget


In [132]:
r0=resultcalculation(calcdf,meandf,stddf, 1 , st_run_weights)

In [133]:
#calcdf['dribbler']=r2

calcdf['2nd runner']=r0
#calcdf['Res2def']=r2

In [136]:
calcdf.to_csv('final_st_rating.csv' , index=False)

In [257]:
ranking=calcdf.sort_values(by='Res1def', ascending=False)

In [57]:
r=pd.read_csv('ranking/final/ranking_st_fa9.csv')

In [58]:
ranking=r.sort_values(by='fa9', ascending=False)

In [59]:
ranking=r.sort_values(by='fa9', ascending=False)
ranking=ranking.reset_index()

In [ ]:
ranking[ranking['playerId']==614755]

In [61]:
bestplayers(ranking)

matchid: 2800011
playerid: 101669
A. Dzyuba
matchid: 5414266
playerid: 614755
Z. Amdouni
matchid: 2799552
playerid: 8717
H. Kane
matchid: 2799965
playerid: 97592
M. Dabbur
matchid: 2799578
playerid: 8717
H. Kane
matchid: 2799923
playerid: 3291
Paco Alcácer
matchid: 5414162
playerid: 273206
G. Rodrigues
matchid: 2799664
playerid: 118
M. Depay
matchid: 2800067
playerid: 14879
T. Pukki
matchid: 2799654
playerid: 3322
Cristiano Ronaldo
matchid: 2800021
playerid: 454072
A. Georgiou
matchid: 2800028
playerid: 101669
A. Dzyuba
matchid: 2799965
playerid: 20824
E. Zahavi
matchid: 2799941
playerid: 14703
M. Arnautović
matchid: 2799660
playerid: 135923
Gonçalo Paciência
matchid: 2799813
playerid: 15199
M. Gavranović
matchid: 2800039
playerid: 20479
F. Quagliarella
matchid: 2799667
playerid: 234710
C. Washington
matchid: 2800049
playerid: 8327
E. Džeko
matchid: 2799838
playerid: 282768
C. Itten
matchid: 5414229
playerid: 3327
Joselu
matchid: 2799587
playerid: 8717
H. Kane
matchid: 2799685
playerid

In [ ]:
path="norm_stats/adjustedstrikerstats.csv"
comparison(101669,2800011,path, meandf.columns)

In [259]:
ranking.to_csv('rank_fb_runweights.csv',index=False)